In [1]:
import tensorflow as tf
import chess
from chess.pgn import read_game
import numpy as np
from anytree import Node
from network import ChessNeuralNetwork
from game import Chess

import time
from agents.nn_agent import NeuralNetworkAgent

ImportError: Traceback (most recent call last):
  File "/Users/adam/anaconda/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/__init__.py", line 49, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "/Users/adam/anaconda/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "/Users/adam/anaconda/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow', fp, pathname, description)
  File "/Users/adam/anaconda/envs/tensorflow/lib/python3.6/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/Users/adam/anaconda/envs/tensorflow/lib/python3.6/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: dlopen(/Users/adam/anaconda/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/_pywrap_tenso

In [3]:
def board_generator():
    pgn = open("millionbase-2.22.pgn")
    while True:
        game = read_game(pgn)
        if game and len(list(game.main_line()))>0:
            node = game
            move_number = np.random.randint(0, high=len(list(game.main_line())) - 1) #don't take the last move
            while 2*(node.board().fullmove_number-1)+int(not node.board().turn) < move_number:
                next_node = node.variation(0)
                node = next_node
            yield node.board()
        else:
            pgn.seek(0)

In [4]:
def simple_value(board):
    values = [1, 3, 3, 5, 9]
    s = 0
    for i, v in enumerate(values):
        s += Chess.pad_bitmask(board.pieces_mask(i + 1, 1)).sum() * v
        s -= Chess.pad_bitmask(board.pieces_mask(i + 1, 0)).sum() * v
    return np.tanh(s / 5)

In [5]:
def alpha_beta(node, depth, alpha, beta):
    if depth == 0 or node.board.is_game_over():
        return simple_value(node.board), node

    legal_moves = list(node.board.legal_moves)
    child_boards = [node.board.copy() for _ in legal_moves]
    children = []
    for idx in range(len(node.board.legal_moves)):
        child_boards[idx].push(legal_moves[idx])
        child = Node(str(legal_moves[idx]), parent=node, board=child_boards[idx], move=legal_moves[idx])
        children.append(child)
    n = node
    if node.board.turn:
        v = -100000
        for child in children:
            vv, nn = alpha_beta(child, depth - 1, alpha, beta)
            if vv > v:
                v = vv
                n = nn
            alpha = max(alpha, v)
            if beta <= alpha:
                break  # (* β cut-off *)
        return v, n
    else:
        v = 100000
        for child in children:
            vv, nn = alpha_beta(child, depth - 1, alpha, beta)
            if vv < v:
                v = vv
                n = nn
            beta = min(beta, v)
            if beta <= alpha:
                break  # (* α cut-off *)
        return v, n

In [6]:
import tensorflow as tf
sess = tf.Session()
network = ChessNeuralNetwork()
agent = NeuralNetworkAgent('master', network, create_trainer=False)
sess.run(tf.global_variables_initializer())

In [7]:
gen = board_generator()
b = gen.__next__()
root = Node('root', board=b, move=None)
t0 = time.time()
v, n = alpha_beta(root, 1, -100000, 100000)
print(time.time() - t0, 'seconds')
print(v, n)

0.012464046478271484 seconds
0.0 Node('/root/b8a6', board=Board('r1bqkbnr/pppppppp/n7/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2'), move=Move.from_uci('b8a6'))


In [9]:
b = gen.__next__()
boards_and_moves = [(b.copy(), move) for move in b.legal_moves]
for board, move in boards_and_moves:
    board.push(move) 
boards = list(zip(*boards_and_moves))[0]
fvs = np.vstack([Chess.make_feature_vector(board) for board in boards])
values = sess.run(network.value, feed_dict={network.feature_vector_: fvs})
if b.turn:
    move_idx = np.argmax(values)
else:
    move_idx = np.argmin(values)

print(boards[move_idx], '\n\n', np.max(values))

env = Chess(b)
move, value = agent.get_move(sess, env, 1)
env.make_move(move)
print(env.board, '\n\n', value)
print(boards[move_idx] == env.board)

b . r r . b . .
p . . . . k . .
. . . . R . . p
. . p p . . B .
. . . . . . . .
. N P . . . . .
P . . . . P P P
. . . . R . K . 

 0.319721
b . r r . b . .
p . . . . k . .
. . . . R . . p
. . p p . . B .
. . . . . . . .
. N P . . . . .
P . . . . P P P
. . . . R . K . 

 0.319721
True


In [7]:
def make_feature_vector(board):
    piece_matrix = np.zeros((64, len(chess.PIECE_TYPES) + 1, len(chess.COLORS)))

    # piece positions
    for piece in chess.PIECE_TYPES:
        for color in chess.COLORS:
            piece_matrix[:, piece, int(color)] = ChessNeuralNetwork.pad_bitmask(board.pieces_mask(piece, color))

    # en passant target squares
    if board.ep_square:
        piece_matrix[board.ep_square, -1, int(board.turn)] = 1

    reshaped_piece_matrix = piece_matrix.reshape((64, (len(chess.PIECE_TYPES) + 1) * len(chess.COLORS)))
    feature_array = np.zeros((64, (len(chess.PIECE_TYPES) + 1) * len(chess.COLORS) + 2))
    feature_array[:, :-2] = reshaped_piece_matrix

    # empty squares
    empty_squares = (reshaped_piece_matrix.sum(axis=1) == 0)
    feature_array[empty_squares, :-2] = 1

    # castling rights
    feature_array[:, -1] = ChessNeuralNetwork.pad_bitmask(board.castling_rights)

    feature_vector = np.zeros((1, 1025))
    feature_vector[0, :-1] = np.reshape(feature_array, (1024,))
    feature_vector[0, -1] = board.turn

    return feature_vector

In [9]:
gen = board_generator()
board = gen.__next__()
feature_vector = ChessNeuralNetwork.make_feature_vector(board)
print(sess.run(network.value, feed_dict={network.feature_vector_: feature_vector})[0,0])
print(simple_value(board))

-0.197375
-0.197375320225


In [10]:
from random import choice, random

def generate_random_fen(density):
    pieces = ['p', 'n', 'b', 'r', 'q', 'P', 'N', 'B', 'R', 'Q']
    rank_fens = []
    for rank in range(8):
        rank_fen = ''
        empty_count = 0
        for file in range(8):
            if random() > density:
                empty_count += 1
            else:
                if empty_count > 0:
                    rank_fen += str(empty_count)
                    empty_count = 0
                piece = choice(pieces)
                rank_fen += piece
        if empty_count > 0:
            rank_fen += str(empty_count)
        rank_fens.append(rank_fen)
    fen = '/'.join(rank_fens)
    
    if random() < .5:
        fen += ' w '
    else:
        fen += ' b '
    casteling_rights = ''
    if random() < .5:
        casteling_rights += 'K'
    if random() < .5:
        casteling_rights += 'Q'
    if random() < .5:
        casteling_rights += 'k'
    if random() < .5:
        casteling_rights += 'q'
    if casteling_rights == '':
        casteling_rights = '--'
    fen += casteling_rights
        
    fen += ' - 0 1'
    return fen

In [11]:
def batch_generator(batch_size):
    while True:
        X = []
        y = []
        for i in range(batch_size):
            fen = generate_random_fen(random())
            board = chess.Board(fen)
            fv = ChessNeuralNetwork.make_feature_vector(board)
            X.append(fv)
            y.append(simple_value(board))
        yield np.array(X), np.array(y)

In [12]:
g = batch_generator(10)
X, y = g.__next__()
print(X)
print(y)

[[[ 0.  0.  0. ...,  0.  1.  1.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]]

 [[ 0.  0.  0. ...,  0.  1.  0.]]

 ..., 
 [[ 0.  0.  0. ...,  0.  1.  0.]]

 [[ 0.  0.  1. ...,  0.  1.  0.]]

 [[ 0.  0.  0. ...,  0.  1.  1.]]]
[-0.99969858  0.9993293   0.99969858  0.37994896 -0.99986455  0.98367486
 -0.97574313 -0.53704957 -0.19737532  0.99263152]


In [22]:
fen = generate_random_fen(.1)
board = chess.Board(fen)
feature_vector = ChessNeuralNetwork.make_feature_vector(board)
print(sess.run(network.value, feed_dict={network.feature_vector_: feature_vector})[0, 0])
print(simple_value(board))

0.761594
0.761594155956


In [23]:
print(board)

. . p . . . . .
. . . . . . . .
. Q B . . . . .
. . . . . . . N
. . . . . . . .
. . . . . . . .
. . . . . q . .
. . . . . . . .


In [24]:
ChessNeuralNetwork.pad_bitmask(board.pieces_mask(2, 1)).reshape((8, 8))

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [17]:
W_1 = np.zeros((1025, 100))
pieces = ['p', 'n', 'b', 'r', 'q', 'P', 'N', 'B', 'R', 'Q']
values = [-1, -3, -3, -5, -9, 1, 3, 3, 5, 9]
for piece, value in zip(pieces, values):
    fen = '/'.join([8*piece for _ in range(8)]) + ' b -- - 0 1'
    board = chess.Board(fen)
    W_1[ChessNeuralNetwork.make_feature_vector(board)[0] == 1, 0] = value

In [25]:
np.dot(feature_vector, W_1)

array([[ 5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [39]:
w = sess.run(network.trainable_variables)[0][:-1, :].reshape(64, -1, 100)\

In [46]:
w[:, 2, 0]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.], dtype=float32)

In [26]:
sess.run(network.value, feed_dict={network.feature_vector_: network.make_feature_vector(board)})

array([[ 0.76159418]], dtype=float32)

In [30]:
np.tanh(5/5)

0.76159415595576485